# RAG Implementation Using
-  Pinecone(managed Vector DB)
-  OpenAI (embedding model)

## Ingesting the data to the vector database

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

load_dotenv()

# 1. Load the Documents (Text file)
loader = TextLoader('./Data/mediumBlog.txt', encoding='UTF-8')
document = loader.load()

#2.split it into chunks
print("splitting...")
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(document)

print(f"created {len(texts)} chunks")

#3.Create Embedding form the splitted text documnt

#Embedding model Initialization
embeddings = OpenAIEmbeddings()


#4. Process and Store in Vector DB
print("Ingestion....")
PineconeVectorStore.from_documents(texts, embeddings, index_name=os.environ["INDEX_NAME"])
print("##--Finish--##")

## RAG using Chain constructors

In [10]:
import os

from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_pinecone import PineconeVectorStore
from langchain import hub

#New imports related to RAG
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.runnables import RunnablePassthrough


load_dotenv()

print("Retrieving...")
#Embedd the Input query
embeddings = OpenAIEmbeddings()

llm = ChatOpenAI()
# query = 'who is the author of the medium blog post related to vector database?'
# query = 'what did Chip Huen quoted'
query = 'when was the blog published?'
prompt = PromptTemplate.from_template(template=query)
chain = prompt | llm
result = chain.invoke({})
print("--------Normal Response without RAG------")
print(result.content)

vectorStore = PineconeVectorStore(embedding=embeddings,index_name=os.environ["INDEX_NAME"])


#Download PROMPT for RETRIEVAL
retrieval_prompt = hub.pull('langchain-ai/retrieval-qa-chat')

#Format list of document into a prompt and pass to llm
combine_doc_chain = create_stuff_documents_chain(llm,retrieval_prompt)

#Retrieve data from the VectorStore (Retriver)
retrieval_chain = create_retrieval_chain(retriever=vectorStore.as_retriever(), combine_docs_chain=combine_doc_chain)

#invoke the retrieval chain
result = retrieval_chain.invoke(input={"input":query})

print("--------using RAG--------")
print(result["answer"])

Retrieving...
--------Normal Response without RAG------
I'm sorry, but without the specific blog post or blog URL, I am unable to provide the exact date when it was published.
--------using RAG--------
The blog was published on December 22, 2023.


## RAG using LCEL chains

In [14]:
import os

from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_pinecone import PineconeVectorStore
from langchain import hub

#New imports related to RAG
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.runnables import RunnablePassthrough


load_dotenv()

# Function formats the document list from vector db -- same as (create_stuff_documents_chain)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


print("Retrieving...")
llm = ChatOpenAI()
embeddings = OpenAIEmbeddings()

# After ingesting the data --> Access the vectorDB using their index_name in DB
vectorStore = PineconeVectorStore(embedding=embeddings,index_name=os.environ["INDEX_NAME"])
    
    
#Using LCEL to build the RAG chain without chain constructor (create_stuff_document_chain & create_retrieval_chain)
template = """
    Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Use three sentences maximum and keep the answer as concise as possible. 
    Always say "thanks for asking!" at the end of the.
  
    {context}
    
    Question:{question}
    
    Helpful Answer:
    """

custom_rag_prompt = PromptTemplate.from_template(template=template)

rag_chain = ({"context":vectorStore.as_retriever() | format_docs,"question":RunnablePassthrough()}
             | custom_rag_prompt
             | llm)

result = rag_chain.invoke("get me author name")

print(result.content)

Retrieving...
The author of the article is Ejiro Onose. Thanks for asking!
